In [47]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Initialize the model and move it to the GPU
model = NeuralNetwork().to(device)

Using device: cuda


In [48]:
learning_rate = 1e-3
batch_size = 20
epochs = 5

In [49]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [50]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [51]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [52]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306843  [   64/60000]
loss: 2.294107  [ 2064/60000]
loss: 2.272974  [ 4064/60000]
loss: 2.266975  [ 6064/60000]
loss: 2.249668  [ 8064/60000]
loss: 2.207595  [10064/60000]
loss: 2.230912  [12064/60000]
loss: 2.181103  [14064/60000]
loss: 2.177765  [16064/60000]
loss: 2.155216  [18064/60000]
Test Error: 
 Accuracy: 38.2%, Avg loss: 2.144791 

Epoch 2
-------------------------------
loss: 2.151001  [   64/60000]
loss: 2.144111  [ 2064/60000]
loss: 2.081663  [ 4064/60000]
loss: 2.104431  [ 6064/60000]
loss: 2.047416  [ 8064/60000]
loss: 1.979669  [10064/60000]
loss: 2.012721  [12064/60000]
loss: 1.920301  [14064/60000]
loss: 1.923919  [16064/60000]
loss: 1.859004  [18064/60000]
Test Error: 
 Accuracy: 52.4%, Avg loss: 1.853492 

Epoch 3
-------------------------------
loss: 1.882970  [   64/60000]
loss: 1.852381  [ 2064/60000]
loss: 1.734275  [ 4064/60000]
loss: 1.784409  [ 6064/60000]
loss: 1.675448  [ 8064/60000]
loss: 1.625861  [10064/600

KeyboardInterrupt: 